# Promote your instagram account

This notebook can be used to automate some of your instagram interactions and stimulate your audience growth. Be aware it is against Instagram's terms of use to use bots and you'll risk getting banned in case of any suspicious activity - so be smart and don't overdo it. 

In [ ]:
from selenium import webdriver #automation library
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd
import numpy as np
import datetime
from time import sleep, strftime
from random import randint
import emoji

In [ ]:
# set up driver
chromedriver_path = '/usr/local/bin/chromedriver' # Change this to your own chromedriver path!
driver = webdriver.Chrome(executable_path=chromedriver_path)
wait = WebDriverWait(driver, 10)

# set globals
USERNAME = 'MyUsername' # change to your username
PASSWORD = 'MyPassword' # change to your password

### Define functions

After loading the required libraries and setting globals, we define 3 functions: 

<ol>
<li>`login()`: loads instagram webpage and logs in. Note that paths might change over time.</li>
<li>`place_comment()`: performs the activity of placing a comment. Comments are placed randomly and these settings can be adjusted in the code if required. </li>
<li>`instaract`: responsible for the instagram interactions follow, like, and comment. Comments are optional, and to be activated using comment argument. </li>
</ol>

In [ ]:
# Please note all paths are hardcoded and might change

def login(username, password):
    driver = webdriver.Chrome(executable_path=chromedriver_path)
    driver.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
    sleep(2)
    
    try:
        accept_cookies = driver.find_element_by_xpath("//div[@class='mt3GC']/button[1]")
        accept_cookies.click()
        sleep(1)
    except:
        pass

    username_field = driver.find_element_by_name('username')
    username_field.send_keys(username)
    password_field = driver.find_element_by_name('password')
    password_field.send_keys(password)

    button_login = driver.find_element_by_xpath("//form[@id='loginForm']/div/div[3]/button")
    button_login.click()
    sleep(3)
    
    try:
        # do not save login info
        notnow = driver.find_element_by_xpath("//div[@class='cmbtv']/button")
        notnow.click() 
    except:
        pass
    try:
        # do not turn on notifications
        notnow = driver.find_element_by_xpath("//div[@class='mt3GC']/button[2]")
        notnow.click()
    except:
        pass
    return driver


In [ ]:
def place_comment(driver, comments):
    """ Leave a comment at random selection of viewed pictures
    """
    prob = randint(1,10)
    print(prob)
    if prob >= 6:
        sleep(3)
        comments += 1  
        if (prob >= 8):
            comment_box = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//form[@class = 'X7cDz']/textarea")))
            sleep(1)
            comment_box.send_keys('Keep it up!' + Keys.RETURN) # change if necessary
        elif (prob > 5) and (prob < 8):
            comment_box = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//form[@class = 'X7cDz']/textarea")))
            sleep(1)
            comment_box.send_keys('Nice!' + Keys.RETURN) # change if necessary

        sleep(randint(5,8))
    return comments
    

def instaract(hashtags, max_range, comment):
    """
    :param hashtags:   (list) hashtags to filter on
    :param follow:     (boolean) follow based on hashtag
    :param like:       (boolean) like based on hashtag
    :param comment:    (boolean) comment based on hashtag
    """
    new_followed = []
    tag = -1
    follows = 0
    likes = 0
    comments = 0
    
    for hashtag in hashtags:
        tag += 1
        driver.get('https://www.instagram.com/explore/tags/'+ hashtags[tag] + '/')
        sleep(3)
        first_thumbnail = driver.find_element_by_xpath("//div[@class = 'v1Nh3 kIKUG  _bz0w']")
        first_thumbnail.click()
        sleep(randint(1,2))    
        try:        
            for x in range(1,max_range):
                insta_user = driver.find_element_by_xpath("//a[@class = 'sqdOP yWX7d     _8A5w5   ZIAjV ']").text

                if insta_user not in prev_user_list:
                    follow_button = driver.find_element_by_xpath("//div[@class = 'bY2yH']/button")
                    if follow_button.text == 'Follow': # if we alread follow, do not unfollow
                        follow_button.click()
                        follows += 1

                    like_button = driver.find_element_by_xpath("//span[@class = 'fr66n']/button")
                    like_button.click()
                    likes +=1
                    sleep(2)
                    
                    if comment:
                        try:
                            comments = place_comment(driver, comments)
                        except Exception as e: print(e)

                    # Next picture
                    driver.find_element_by_link_text('Next').click()
                    sleep(randint(5,10))
                else:
                    driver.find_element_by_link_text('Next').click()
                    sleep(randint(5,10))

        # some hashtag stops refreshing photos (it may happen sometimes), it continues to the next
        except:
            print('faal')
            continue
    return new_followed, follows, likes, comments


### Get promoted

Define a list of hashtags that contain posts related to the content you're creating on your page, to target the audience you would like to interact with. If required, you can track a list of previously followed users, in case you would like to unfollow them after a couple of days

In [ ]:
hashtags = ['somehashtagyoulike', 'anotherhashtagyoulike'] # update list to your own preferences

prev_user_list = [] # if it's the first time you run it, use this line and comment the two below
# prev_user_list = pd.read_csv('<insert_date>_users_followed.csv', delimiter=',').iloc[:,1:2] # useful to build a user log
# prev_user_list = list(prev_user_list['0'])

In [ ]:
driver = login(USERNAME, PASSWORD)
new_followed, follows, likes, comments = instaract(hashtags, 5, comment=True)

for n in range(0,len(new_followed)):
    prev_user_list.append(new_followed[n])
    
updated_user_df = pd.DataFrame(prev_user_list)
updated_user_df.to_csv('{}_users_followed.csv'.format(strftime("%Y%m%d-%H%M%S")))
print('Liked {} photos.'.format(likes))
print('Commented {} photos.'.format(comments))
print('Followed {} new people.'.format(follows))